This notebook will assess a variety of ML algorithms in their performance in predicting house prices. We'll be considering lasso and ridge regression, random forests, xgboost and maybe later light gbm. All models will be tuned via Bayesian Optimisation to minimise the average 5-fold cross validation score. Note that here specifically we are predicting log house prices and using the RMSE metric so our model evaluation will need to be tailored to that.

First we'll import the data and clean it so that it can be used in a ML friendly format. We'll use a similar process to the one in the EDA notebook. Since exploratory data analysis has already been covered in the other notebook, we won't bother repeating it here.

In [1]:
import pandas as pd
import numpy as np
import time
from scipy.stats import skew
from bayes_opt import BayesianOptimization

In [2]:
train = pd.read_csv('Data/train.csv')
test = pd.read_csv('Data/test.csv')

#Stripping SalePrice from the training data and combining with the test data
SalePrice = np.log(train['SalePrice'])
train = train.drop('SalePrice',axis=1)
X = pd.concat([train,test], ignore_index=True)

Doing some data pre-processing

In [3]:
#MSSubClass is a categorical variable, so we convert it to a string 
X['MSSubClass'] = X['MSSubClass'].apply(str)
X['LotFrontage'] = X['LotFrontage'].fillna(0)
train['GarageYrBlt'] = train['GarageYrBlt'].fillna(0) 

In [4]:
##Dealing with numerics
numerics = X.select_dtypes(exclude='object')

#Filling nas
numerics = numerics.fillna(method='ffill')

#log transformation of skewed variables
skews = numerics.apply(lambda x:skew(x.dropna()))
skewed = skews>0.75
skewed_data = numerics[skewed.index[skewed]]
skewed_feats = skewed_data.columns
numerics = numerics.drop(skewed_feats, axis=1)
log_transformed = np.log1p(skewed_data)
numerics = pd.concat([numerics,log_transformed],axis=1)


In [5]:
#Dealing with strings
strings = X.select_dtypes(include='object')

#Converting strings to dummies and joining with numerics
dummies = pd.get_dummies(strings)
X = pd.concat([numerics,dummies],axis=1)

#Splitting into train and test data
X_train = X.iloc[:train.shape[0],]
X_test = X.iloc[train.shape[0]:,]
train = pd.concat([SalePrice,X_train],axis=1)
print(X_train.shape, X_test.shape)
print(train.shape, test.shape)

(1460, 304) (1459, 304)
(1460, 305) (1459, 80)


So now that the initial data preprocessing has been done, we are now going to set up cross-validation and Bayesian optimisation procedures for each model. The reason why we use Bayesian Optimisation for hyperparameter tuning is because of the stochastic nature of our objective function. We are randomly sorting the data to better replicate the variability in the dgp when measuring the validation score. While a deterministic optimiser may work, it may not be the best solution if it does not account for the noise.

In [20]:
#Setting up optimisation for Ridge regression model
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge

def ridge_score(alpha):
    train_data = train.sample(frac=1)
    X=train_data.iloc[:,1:]
    y=train_data['SalePrice']
    
    scores = cross_val_score(estimator=Ridge(alpha=alpha), X=X, y=y, scoring='neg_mean_squared_error', cv=5)
    return(-np.average(np.sqrt(-scores)))

bounds_ridge = {'alpha': (0,20)}

For the parameter bounds, I have chosen alpha to be between 0 and 20. Using a 0 value will be equivalent to estimation by OLS, any positive values will shrink the parameter estimates towards zero. I do not want to using an upper bound that is too high as it will waste computation power as in practice, optimal values for alpha are relatively small.

In [26]:
#Running Bayes Opt to tune ridge regression
BO = BayesianOptimization(ridge_score, bounds_ridge)

BO.probe({"alpha":10},lazy=False)
BO.maximize(n_iter=50,alpha=0.0001)

|   iter    |  target   |   alpha   |
-------------------------------------
|  1        | -0.1329   |  63.23    |
|  2        | -0.1284   |  13.79    |
|  3        | -0.133    |  75.6     |
|  4        | -0.1352   |  95.52    |
|  5        | -0.1389   |  155.2    |
|  6        | -0.1407   |  200.0    |
|  7        | -1.658e+1 |  0.0      |
|  8        | -0.1297   |  11.57    |
|  9        | -0.1282   |  10.57    |
|  10       | -0.1257   |  13.01    |
|  11       | -0.1266   |  13.47    |
|  12       | -0.1285   |  10.24    |
|  13       | -0.1259   |  13.62    |
|  14       | -0.1268   |  10.17    |
|  15       | -0.1281   |  13.63    |
|  16       | -0.1302   |  10.16    |
|  17       | -0.1281   |  13.63    |
|  18       | -0.1293   |  10.15    |
|  19       | -0.1313   |  11.1     |
|  20       | -0.1277   |  12.64    |
|  21       | -0.1301   |  10.15    |
|  22       | -0.1293   |  13.64    |
|  23       | -0.1309   |  10.15    |
|  24       | -0.1266   |  13.64    |
|  25       

The optimal choice of alpha seems to be around 10-20. Though depending on how the optimiser progresses, it can sometimes get stuck near the upper bound. I had to use the probe method to ensure that the optimizer started at a good value, otherwise it can get stuck on the upperbound.

In [27]:
#Setting up optimisation for Lasso model
from sklearn.linear_model import Lasso

def lasso_score(alpha):
    train_data = train.sample(frac=1)
    X=train_data.iloc[:,1:]
    y=train_data['SalePrice']

    scores = cross_val_score(estimator=Lasso(alpha=alpha, max_iter=10000), X=X, y=y, scoring='neg_mean_squared_error', cv=5)
    return(-np.average(np.sqrt(-scores)))

bounds_lasso = {'alpha': (0.00001,20)}

I'm adjusting the bounds for the lasso as 0 values for regularisation cause convergence problems.

In [30]:
#Running Bayes Opt to tune Lasso regression

BO = BayesianOptimization(lasso_score, bounds_lasso)

BO.probe({"alpha":0.001},lazy=False)
BO.maximize(n_iter=50,alpha=0.0001)

|   iter    |  target   |   alpha   |
-------------------------------------
|  1        | -0.3077   |  10.5     |
|  2        | -0.3068   |  11.0     |
|  3        | -0.3131   |  16.5     |
|  4        | -0.3041   |  4.121    |
|  5        | -0.3163   |  19.36    |
|  6        | -0.1382   |  1e-05    |
|  7        | -0.14     |  1e-05    |
|  8        | -0.1407   |  1e-05    |
|  9        | -0.137    |  1e-05    |
|  10       | -0.137    |  1e-05    |
|  11       | -0.1405   |  1e-05    |
|  12       | -0.1385   |  1e-05    |
|  13       | -0.1387   |  1e-05    |
|  14       | -0.1387   |  1e-05    |
|  15       | -0.1387   |  1e-05    |
|  16       | -0.1358   |  1e-05    |
|  17       | -0.139    |  1e-05    |
|  18       | -0.1356   |  1.001e-0 |
|  19       | -0.1325   |  1e-05    |
|  20       | -0.1325   |  1e-05    |
|  21       | -0.1325   |  1e-05    |
|  22       | -0.1325   |  1e-05    |
|  23       | -0.1325   |  1e-05    |
|  24       | -0.1325   |  1e-05    |
|  25       

I'm quite suspicious about the behaviour of the Bayes optimiser in this case, it cannot seem to explore other values than the lower bound. While I could change the lower bound, it will cause convergence issues and will require an increased number of iterations. I can handpick a value for alpha that does noticeably better.

In [31]:
lasso_score(0.0001)

-0.12852462107663287

At this point I'm not going to try delve into the workings of optimiser as I don't think it will be very fruitful. Instead I'm going to try using a different optimiser to see if we get better convergence.

In [32]:
from scipy.optimize import minimize

minimize(lasso_score, x0=1, bounds = ((0.00001,20),), tol=0.00001)

      fun: -0.26752613266792663
 hess_inv: <1x1 LbfgsInvHessProduct with dtype=float64>
      jac: array([-213127.13829123])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 50
      nit: 2
   status: 0
  success: True
        x: array([0.99999921])

This also performs quite poorly. I think it gets thrown off by the randomness of the objective function. It does find a good optimum, but only if I put in an optimal starting point!

In [33]:
minimize(lasso_score, x0=0.001, bounds = ((0.00001,20),), tol=0.00001)

      fun: -0.1276295819828786
 hess_inv: <1x1 LbfgsInvHessProduct with dtype=float64>
      jac: array([89516.17478705])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 36
      nit: 2
   status: 0
  success: True
        x: array([0.00099648])

Both ridge and lasso seem to perform quite well.

In [34]:
#Creating optimizer for a Random Forests regressor
from sklearn.ensemble import RandomForestRegressor as RF

def RF_score(n_estimators,max_depth,min_samples_split,min_samples_leaf,max_features):
    
    #Contraining hyperparameters to be converted to integers (e.g. number of decision trees can't be continuous!)
    n_estimators = int(n_estimators)
    max_depth = int(max_depth)
    min_samples_split = int(min_samples_split)
    min_samples_leaf = int(min_samples_leaf)
    max_features = int(max_features)
    
    assert type(n_estimators) == int
    assert type(max_depth) == int
    assert type(min_samples_split) == int
    assert type(min_samples_leaf) == int
    assert type(max_features) == int
    
    train_data = train.sample(frac=1)
    X=train_data.iloc[:,1:]
    y=train_data['SalePrice']

    scores = cross_val_score(
        estimator=RF(
                    n_estimators=n_estimators, 
                    max_depth=max_depth, 
                    min_samples_split=min_samples_split,
                    min_samples_leaf = min_samples_leaf,
                    max_features = max_features),
    X=X, y=y, scoring='neg_mean_squared_error', cv=5)
    return(-np.average(np.sqrt(-scores)))

bounds_RF = {
    'n_estimators': (1,3000),
    'max_depth': (1,100),
    'min_samples_split': (2,200),
    'min_samples_leaf': (1,200),
    'max_features': (1,290)
}

In [ ]:
#Running Bayes Opt to tune Rando Forests regression
BO = BayesianOptimization(RF_score, bounds_RF)

BO.probe({'n_estimators':20, 
          'max_depth':100, 
          'min_samples_split':2,
          'min_samples_leaf': 1,
          'max_features': 130},
        lazy = False)

BO.maximize(n_iter=50,alpha=0.0001)

|   iter    |  target   | max_depth | max_fe... | min_sa... | min_sa... | n_esti... |
-------------------------------------------------------------------------------------
|  1        | -0.1793   |  6.65     |  278.8    |  13.95    |  99.06    |  1.956e+0 |
|  2        | -0.2507   |  27.67    |  219.0    |  165.7    |  41.0     |  1.803e+0 |
|  3        | -0.1568   |  91.48    |  112.6    |  10.95    |  48.4     |  1.858e+0 |
|  4        | -0.1627   |  14.49    |  124.5    |  2.196    |  71.42    |  2.041e+0 |
|  5        | -0.1947   |  78.26    |  95.5     |  70.24    |  148.3    |  2.381e+0 |
|  6        | -0.2829   |  63.24    |  2.142    |  3.485    |  192.6    |  804.0    |
|  7        | -0.148    |  100.0    |  290.0    |  1.0      |  2.0      |  3e+03    |
|  8        | -0.1658   |  22.58    |  10.69    |  3.329    |  10.52    |  2.974e+0 |
|  9        | -0.1721   |  81.14    |  287.1    |  11.62    |  68.91    |  8.593    |
|  10       | -0.1418   |  79.81    |  185.9    |  2.2

Seems that the Random Forest algorithm doesn't do as well as simple regularised linear models. It may be able to perform better with a different tuning protocol, perhaps something to explore in another notebook?

In [ ]:
#Creating optimiser for xgboost, note that doing it this way isn't entirely necessary
#xgboost already contains the inbuilt functionality to do so, but I want to be consistent in my approach
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import xgboost as xgb

def xgb_score(eta, max_depth, gamma, colsample, subsample, early_stop):
    
    #Contraining hyperparameters to be converted to integers (e.g. number of decision trees can't be continuous!)
    max_depth = int(max_depth)
    early_stop = int(early_stop)
    
    #assert type(n_estimators) == int
    assert type(max_depth) == int
    assert type(early_stop) == int
    
    train_data = train.sample(frac=1)
    X=train_data.iloc[:,1:]
    y=np.array(train_data['SalePrice'])
    

    xgb_model = xgb.XGBRegressor(learning_rate=0.1, 
                                 max_depth=max_depth, 
                                 min_split_loss=gamma, 
                                 colsample_bytree = colsample,
                                 subsample = subsample,
                                 early_stopping_rounds = early_stop,
                                 objective="reg:squarederror")
    
    scores = cross_val_score(estimator=xgb_model, X=X, y=y, scoring='neg_mean_squared_error', cv=5)
    return(-np.average(np.sqrt(-scores)))

bounds_xgb = {'eta': (0.01,0.5),
              'max_depth':(3,100),
              'gamma':(0,0.4), 
              'colsample':(0.3,1), 
              'subsample':(0.3,1),
              'early_stop':(1,15)}

In [16]:
#Running Bayes Opt to tune Random Forests regression

BO = BayesianOptimization(xgb_score, bounds_xgb)
BO.maximize(n_iter=50,alpha=0.0001)

|   iter    |  target   | colsample | early_... |    eta    |   gamma   | max_depth | subsample |
-------------------------------------------------------------------------------------------------
|  1        | -0.1421   |  0.9163   |  8.453    |  0.07675  |  0.3335   |  19.24    |  0.9383   |
|  2        | -0.1381   |  0.4751   |  11.29    |  0.118    |  0.2687   |  11.04    |  0.7378   |
|  3        | -0.1326   |  0.7861   |  13.81    |  0.08371  |  0.07473  |  21.65    |  0.8461   |
|  4        | -0.1305   |  0.7299   |  9.31     |  0.4099   |  0.1261   |  27.06    |  0.3844   |
|  5        | -0.1388   |  0.3472   |  1.883    |  0.03803  |  0.2748   |  15.2     |  0.9554   |
|  6        | -0.1369   |  0.5111   |  1.219    |  0.2971   |  0.002935 |  39.98    |  0.3715   |
|  7        | -0.1398   |  0.3249   |  15.0     |  0.3365   |  0.3099   |  39.91    |  0.4689   |
|  8        | -0.1309   |  0.9567   |  1.13     |  0.2592   |  0.02828  |  3.003    |  0.7279   |
|  9        | -0.133

|  51       | -0.1326   |  0.5703   |  1.037    |  0.4504   |  0.1016   |  3.023    |  0.9569   |
|  52       | -0.1291   |  0.504    |  14.95    |  0.3867   |  0.0374   |  3.018    |  0.7305   |
|  53       | -0.1404   |  0.8258   |  14.99    |  0.3848   |  0.3104   |  3.009    |  0.4784   |
|  54       | -0.1383   |  0.5643   |  1.075    |  0.2045   |  0.1931   |  39.98    |  0.7325   |
|  55       | -0.1428   |  0.5509   |  15.0     |  0.176    |  0.3965   |  3.013    |  0.7466   |


So now that we've tuned the hyperparameters using Bayes opt and gotten a good feel of how the models perform using different parameter sets, it's pretty easy to see that xgboost performs the best. As a final evaluation, I'll put in some paramter values to evaluate our models concurrently and then pick from there.

In [19]:
print(
    lasso_score(0.001),
    ridge_score(10),
    RF_score(n_estimators=20, 
                    max_depth=100, 
                    min_samples_split=2,
                    min_samples_leaf = 1,
                    max_features = 130),
    xgb_score(eta=0.1,
              max_depth=40,
              gamma=0.05,
              colsample=0.9,
              subsample=0.7,
              early_stop=10)
    )

-0.1269655787503537 -0.12987026651616576 -0.14191146053416806 -0.1309665650518888


As noted, xgboost clearly outperforms all other models. So we'll use that configuration to make our test predictions and submit on Kaggle.

In [156]:
model_xgb = xgb.XGBRegressor(learning_rate=0.1, max_depth=10, min_split_loss=0, objective="reg:squarederror")
model_xgb.fit(X_train,SalePrice)
xgb_preds = np.expm1(model_xgb.predict(X_test))


#Submission csv
submission = pd.read_csv('Data/sample_submission.csv')
submission['SalePrice'] = xgb_preds
submission.to_csv('Data/submission.csv',index=False)

/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


Well! Turns out we did rather poorly despite all the work we put in, we got a public score of 0.14504 placing us in the bottom 50% :(.

In [52]:
model_lasso = Lasso(alpha=0.001)
model_lasso.fit(X_train,SalePrice)
lasso_preds = np.expm1(model_lasso.predict(X_test))

#Submission csv
submission = pd.read_csv('Data/sample_submission.csv')
submission['SalePrice'] = lasso_preds
submission.to_csv('Data/submission.csv',index=False)